In [ ]:
import importlib
from matplotlib import pyplot as plt

import astrocast.reduction as red
import astrocast.clustering as clust
import astrocast.analysis as ana
import astrocast.autoencoders as AE
import astrocast.helper as helper

for pack in [red, clust, helper, ana, AE]:
    importlib.reload(pack)

## Create synthetic dataset

In [ ]:
num_signal_types = 32
num_signals = num_signal_types * 10

# Creating signal generators
signal_generators = [helper.SignalGenerator(trace_length=100,
                                            parameter_fluctuations=0) for i in range(num_signal_types)]

# create Events instance
dg = helper.DummyGenerator(generators=signal_generators, num_rows=num_signals)
eObj = dg.get_events()
eObj.name = "original"
display(eObj)

# Plotting
plot = ana.Plotting(events=eObj)
_ = plot.plot_traces(num_samples=16, figsize=(8, 3))

## Filter events 

In [ ]:
eObj_filtered = eObj.copy()

#  that are at least 3 frames and at most 20 frames long
_ = eObj_filtered.filter({"dz": (3, 20)}, inplace=True)
eObj_filtered.name = "filtered"
plot_filtered = ana.Plotting(events=eObj_filtered)

# plot results
_, (ax0, ax1) = plt.subplots(1, 2, sharey=True, figsize=(5, 3))

_ = plot.plot_distribution(column="dz", axx=ax0)
_ = plot_filtered.plot_distribution(column="dz", axx=ax1)
_ = ax0.set_title(eObj.name)
_ = ax1.set_title(eObj_filtered.name)

## Extend events

In [ ]:
#  by 3 events in z0 and z1 direction
#  Note: this feature does not work with synthetic data
# _ = eObj.get_extended_events(use_footprint=True, extend=3, 
#                              in_place=True, load_to_memory=True)

## Enforce length

In [ ]:
# eObj_enforced = eObj.copy()
# eObj_enforced.name = "enforced"

#  enforce and event length of 15
#  Note: this feature does not work with synthetic data
# _ = eObj_enforced.get_extended_events(ensure_min=15, ensure_max=15)

## Normalize

In [ ]:
eObjects = [eObj]

# example 1
#  1) subtract minimum
#  2) divide by maximum

instructions = {
    0: ["subtract", {"mode": "min"}],
    1: ["divide", {"mode": "max"}]
    }

eObj_min_max = eObj.copy()
eObj_min_max.name = "min/max"
_ = eObj_min_max.normalize(instructions, inplace=True)
eObjects.append(eObj_min_max)

# example 2
#  1) calculate the 1st order differential
#  2) divide by the median standard deviation of all events

instructions = {
    0: ["diff", {}],
    1: ["divide", {"mode": "std_median", "population_wide": True}]
    }

eObj_diff = eObj.copy()
eObj_diff.name = "1st order differential"
_ = eObj_diff.normalize(instructions, inplace=True)
eObjects.append(eObj_diff)

# Plot results
_, axx = plt.subplots(1, len(eObjects), figsize=(len(eObjects) * 3, 3), sharex=True, sharey=False)
for i, i_eObj in enumerate(eObjects):
    plot_diff = ana.Plotting(i_eObj.events)
    _ = plot_diff.plot_traces(num_samples=8, ax=axx[i])
    axx[i].set_title(i_eObj.name)

# continue analysis with example 2
eObj = eObj_diff
